In [1]:
from datasets import load_dataset

# dataset = load_dataset("dair-ai/emotion")
dataset = load_dataset("imsoumyaneel/sentiment-analysis-llama2")
dataset

/home/xd/Documents/DS_ML_AL/ML_Notebooks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'text'],
        num_rows: 598298
    })
})

In [2]:
import pandas as pd

complete_dataset = dataset['train'].to_pandas()
complete_dataset['new_label'] = complete_dataset['label'].map({'joy': '0', 'neutral': '1', 'sadness': '2', 'anger': '3', 'fear': '4', 'love': '5', 'surprise': '6'}).values

train_dataset = complete_dataset.sample(frac=0.8,random_state=200)
test_dataset = complete_dataset.drop(train_dataset.index)

complete_dataset.info()
train_dataset.info()
test_dataset.info()

complete_dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598298 entries, 0 to 598297
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentence   598298 non-null  object
 1   label      598298 non-null  object
 2   text       598298 non-null  object
 3   new_label  598298 non-null  object
dtypes: object(4)
memory usage: 18.3+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 478638 entries, 352227 to 559736
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentence   478638 non-null  object
 1   label      478638 non-null  object
 2   text       478638 non-null  object
 3   new_label  478638 non-null  object
dtypes: object(4)
memory usage: 18.3+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 119660 entries, 0 to 598297
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentence   119660 non-null  

,sentence,label,text,new_label
0,I'll throw out the garbage .,neutral,###Human:\nyou are a sentiment analist. guess ...,1
1,"So Dick , how about getting some coffee for to...",joy,###Human:\nyou are a sentiment analist. guess ...,0
2,"Come on , you can at least try a little , besi...",neutral,###Human:\nyou are a sentiment analist. guess ...,1
3,What ’ s wrong with that ? Cigarette is the th...,anger,###Human:\nyou are a sentiment analist. guess ...,3
4,"Not for me , Dick .",neutral,###Human:\nyou are a sentiment analist. guess ...,1


In [3]:
# imports for model creation
import tensorflow as tf
from keras import layers
from keras import losses
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-05-24 01:26:36.647445: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-24 01:26:36.656031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748030196.669125   46242 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748030196.672735   46242 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748030196.682406   46242 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(complete_dataset['sentence'])

vocab_size = len(tokenizer.word_index) + 1
max_length = 200    # max words in a sentence
embedding_dim = 50    # TODO: need to adjust accordinglys

X_train = tokenizer.texts_to_sequences(train_dataset['sentence'])
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')

X_test = tokenizer.texts_to_sequences(test_dataset['sentence'])
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

y_train = train_dataset['new_label']
y_test = test_dataset['new_label']

from keras.utils import to_categorical

num_classes = 7  # Assuming you have 3 classes
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)

In [5]:
labels = complete_dataset['label']

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(478638, 200)
(119660, 200)
(478638,)
(119660,)


In [6]:
# Build the model
model = keras.Sequential([
  keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_shape=(max_length,)),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(7, activation='sigmoid')
])

/home/xd/Documents/DS_ML_AL/ML_Notebooks/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1748030208.588223   46242 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1135 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [7]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# split the dataset into train and test
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(, labels, test_size=0.3, random_state=42, shuffle=True)
# X_train

In [9]:
# train the model
model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test, y_test_encoded))

Epoch 1/10


I0000 00:00:1748030213.379971   46369 service.cc:152] XLA service 0x75f89c0193f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748030213.379984   46369 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-05-24 01:26:53.394941: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748030213.462214   46369 cuda_dnn.cc:529] Loaded cuDNN version 90300


  110/14958 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.2392 - loss: 1.8029 

I0000 00:00:1748030214.909870   46369 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14958/14958 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.5072 - loss: 1.3294 - val_accuracy: 0.8212 - val_loss: 0.5658
Epoch 2/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8458 - loss: 0.4859 - val_accuracy: 0.8601 - val_loss: 0.4155
Epoch 3/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8613 - loss: 0.4204 - val_accuracy: 0.8345 - val_loss: 0.5050
Epoch 4/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8654 - loss: 0.3890 - val_accuracy: 0.8461 - val_loss: 0.4425
Epoch 5/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8709 - loss: 0.3630 - val_accuracy: 0.8660 - val_loss: 0.3744
Epoch 6/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8729 - loss: 0.3469 - val_accuracy: 0.8663 - val_loss: 0.3807
Epoch 7/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8743 - loss: 0.3368 - val_accuracy: 0.8658 - val_loss: 0.3438
Epoch 8/10
14958/14958 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8769 - lo

In [10]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_encoded)
accuracy

3740/3740 ━━━━━━━━━━━━━━━━━━━━ 2s 638us/step - accuracy: 0.8599 - loss: 0.4342


0.8609894514083862

In [11]:
# save the model
import os
try:
    model.save("../models/sentimental-analysis-llama2.keras")
except FileNotFoundError:
    os.mkdir("../models")
    model.save("../models/sentimental-analysis-llama2.keras")

In [22]:
def preprocess_text(text):
    # Tokenize the text
    tokenized_text = tokenizer.texts_to_sequences([text])
    # Pad sequences to the same length as training data
    padded_text = pad_sequences(tokenized_text, maxlen=max_length, padding='post')
    return padded_text

# Preprocess the custom input text
preprocessed_text = preprocess_text("this is good")

# Make predictions
predictions = model.predict(preprocessed_text)

predictions

1/1 [==============================] - 0s 17ms/step


array([[0.809063  , 0.78246254, 0.02547726, 0.03657908, 0.00648503,
        0.02069169, 0.07264358]], dtype=float32)